In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
#Reading The Dataset
df=pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")

In [ ]:
#Learning about the Dataset
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
print(df["Transmission"].unique())
print(df["Seller_Type"].unique())
print(df["Owner"].unique())

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
f_df=df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
f_df.head()

In [ ]:
f_df["Current_year"]=2021
f_df.head()

In [ ]:
#creating a new feture with the help of another one 
f_df["No_year"]=f_df["Current_year"]-f_df["Year"]
f_df.head()

In [ ]:
#Drooping some features which are already been explained
f_df.drop(["Year"],axis=1,inplace=True)
f_df.drop(["Current_year"],axis=1,inplace=True)

In [ ]:
#trying to see what that df is looking like now
f_df.head()

In [ ]:
#creating dummy variable for categorical variables
f_df=pd.get_dummies(f_df,drop_first=True)

In [ ]:
f_df.head()

In [ ]:
#correlation Matrix
f_df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(f_df)

In [ ]:
#Trying to visualize the relationship among the variables
top_corr_features=f_df.corr().index
plt.figure(figsize=(20,20))
g=sns.heatmap(f_df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
#Differentiating the features and explanetory variables
x=f_df.iloc[:,1:]
y=f_df.iloc[:,0]

In [ ]:
y.head()

In [ ]:
x.head()

In [ ]:
#splitting the data into train and test set
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(x,y,test_size=0.2)

In [ ]:
xte.info()

In [ ]:
#some more required packages
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from scipy import stats
from sklearn import svm
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#linear model
modlr=LinearRegression()
modlr=modlr.fit(xtr,ytr)

In [ ]:
#intercept
modlr.intercept_

In [ ]:
#Coeffcients
modlr.coef_

In [ ]:
#R^2 On training data
rsq=modlr.score(xtr,ytr)
rsq

In [ ]:
#residual
predytr=modlr.predict(xtr)
res=ytr-predytr

In [ ]:
#visualizing the residuals are normal or not
stats.probplot(res,plot=plt)
plt.grid()
plt.show()

In [ ]:
#Normality test
stats.normaltest(res)

In [ ]:
#plotting Residual Vs Predicted plot
plt.scatter(predytr,res)
plt.title("Residuals VS Predicted Plot")
plt.show()


Though the normality tests are showing that the residuals are normal but on plotting its not arbitrary so it shows a cluster kind of picture

In [ ]:
#plotting the error on prediction
plr=modlr.predict(xte)
sns.distplot(yte-plr)

In [ ]:
plt.scatter(yte,plr)

In [ ]:
#R^2 on test set
rsq=modlr.score(xte,yte)
rsq

In [ ]:
#Decision tree
modrt=tree.DecisionTreeRegressor(min_samples_split=40)
modrt=modrt.fit(xtr,ytr)

In [ ]:
#param grid
paramgrid=[{"min_samples_split":[35,40,45,50,55,60,65,70,75,80,85,90,95,100,200,300]}]


In [ ]:
search=GridSearchCV(modrt,paramgrid,scoring="neg_mean_squared_error",cv=20,verbose=True)
search=search.fit(xtr,ytr)

In [ ]:
search.best_params_

In [ ]:
modrt=tree.DecisionTreeRegressor(min_samples_split=35)
modrt=modrt.fit(xtr,ytr)

In [ ]:
rsq=modrt.score(xtr,ytr)
rsq

In [ ]:
prt=modrt.predict(xte)
sns.distplot(yte-prt)

In [ ]:
plt.scatter(yte,prt)

In [ ]:
rsq=modrt.score(xte,yte)
rsq

In [ ]:
#plotting the tree
plt.figure(figsize=(20,20))
tree.plot_tree(modrt,feature_names=xte.columns,filled=True)
plt.show()

In [ ]:
#Random Forest
modb=RandomForestRegressor(n_estimators=500,min_samples_split=40,max_features=None)
modb=modb.fit(xtr,ytr)
paramgrid=[{"min_samples_split":[35,40,45,50,55,60,65,70,75,80,85,90,95,100,200,300],"n_estimators":[450,500,550,600,650,700]}]
search=GridSearchCV(modb,paramgrid,scoring="neg_mean_squared_error",cv=3,verbose=True)
search=search.fit(xtr,ytr)

In [ ]:
search.best_params_

In [ ]:
modb=RandomForestRegressor(n_estimators=600,min_samples_split=35,max_features=None)
modb=modb.fit(xtr,ytr)

In [ ]:
rsq=modb.score(xtr,ytr)
rsq

In [ ]:
prf=modb.predict(xte)
sns.distplot(yte-prf)

In [ ]:
plt.scatter(yte,prf)

In [ ]:
rsq=modb.score(xte,yte)
rsq

In [ ]:
#KNN moodel
modknn=KNeighborsRegressor(n_neighbors=5)
modknn=modknn.fit(xtr,ytr)
paramgrid=[{"n_neighbors":[3,4,5,6,7,8,9,10,11,12,13]}]
search=GridSearchCV(modknn,paramgrid,scoring="neg_mean_squared_error",cv=20,verbose=True)
search=search.fit(xtr,ytr)

In [ ]:
search.best_params_

In [ ]:
modknn=KNeighborsRegressor(n_neighbors=13)
modknn=modknn.fit(xtr,ytr)

In [ ]:
rsq=modknn.score(xtr,ytr)
rsq

In [ ]:
pknn=modknn.predict(xte)
sns.distplot(yte-pknn)

In [ ]:
plt.scatter(yte,pknn)

The error shows us that this is not normal

In [ ]:
rsq=modknn.score(xte,yte)
rsq

so, its clear that this model is very very poor

In [ ]:
#SVM
modsvm=svm.SVR()
modsvm=modsvm.fit(xtr,ytr)
paramgrid=[{"kernel":['linear'],"C":[0.1,1]}]
search=GridSearchCV(modsvm,paramgrid,scoring="neg_mean_squared_error",cv=2,verbose=True,n_jobs=-1)
search=search.fit(xtr,ytr)

In [ ]:
search.best_params_

In [ ]:
modsvm=svm.SVR(kernel="linear",C=0.1,degree=1)
modsvm=modsvm.fit(xtr,ytr)

In [ ]:
rsq=modsvm.score(xtr,ytr)
rsq

In [ ]:
modsvm=svm.SVR()
modsvm=modsvm.fit(xtr,ytr)
paramgrid=[{"kernel":['polynomial','rbf'],"C":[0.1,1,5],'degree':[1,2,3]}]
search=GridSearchCV(modsvm,paramgrid,scoring="neg_mean_squared_error",cv=2,verbose=True,n_jobs=-1)
search=search.fit(xtr,ytr)

In [ ]:
search.best_params_

In [ ]:
modsvm=svm.SVR(kernel="rbf",C=0.1,degree=1)
modsvm=modsvm.fit(xtr,ytr)
rsq=modsvm.score(xtr,ytr)
rsq

In [ ]:
#Gradient Boost
clf = GradientBoostingRegressor()
clf.fit(xtr,ytr)
rsq=clf.score(xtr,ytr)
rsq

In [ ]:
predicted = clf.predict(xte)
sns.distplot(yte-predicted)

In [ ]:
plt.scatter(yte,predicted)

In [ ]:
rsq=clf.score(xte,yte)
rsq

In [ ]:
#xgboost
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(xtr,ytr)
rsq=xg_reg.score(xtr,ytr)
rsq

In [ ]:
preds = xg_reg.predict(xte)
sns.distplot(yte-preds)

In [ ]:
plt.scatter(yte,preds)

In [ ]:
rsq=xg_reg.score(xte,yte)
rsq

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(yte, preds))
rmse

In [ ]:
data_dmatrix = xgb.DMatrix(data=xtr,label=ytr)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)
xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 50]
plt.show()